In [1]:
% load_ext autoreload
% autoreload 2


In [2]:
import glob
import os
import numpy as np
import pandas as pd
from collections import defaultdict
import datetime
import src.mex_helper as mex
import folium
from src.utils.map_vis import time_slider_choropleth, geojson_per_row

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import src.utils.gis as gis
from src.features import *

In [5]:
def grid_avgerage(tw_avg, t2g):
    # there are grids without any call throughout the observation period
    g_avg = t2g.merge(tw_avg, left_on='gtid', right_index=True, how='left')

    for h in range(24):
        h=str(h)
        g_avg[h] = g_avg[h] * g_avg['weight']

    g_avg= g_avg.drop(['gtid','weight'], axis=1).groupby('grid').sum()  # fillna=0 by default

    return g_avg

In [6]:
aver_out = mex.stat_tw_dow_aver_hr_uniq_user('out')
tw_avg_wd = pd.DataFrame(aver_out['wd']).T
tw_avg_wk = pd.DataFrame(aver_out['wk']).T

mex_t2g = mex.tower2grid('cities', 1000)
print('number of towers in cities has no call at all during weekday and weekend', len(set(mex_t2g.gtid) - set(tw_avg_wd.index)), len(set(mex_t2g.gtid) - set(tw_avg_wk.index)))


loading cached tw average stats/stat_tw_dow_aver_hr_uniq_user-out.pickle
reading existing t2g file: data/mex_tower/mex_t2g_cities_1000m.csv
number of towers in cities has no call at all during weekday and weekend 233 234


In [7]:
# g_avg = pd.DataFrame([average number of calls], index=grid, columns='hour')
g_avg_wd = grid_avgerage(tw_avg_wd, mex_t2g)
g_avg_wk = grid_avgerage(tw_avg_wk, mex_t2g)

In [46]:
%%time
dv_cities = urban_dilatation_index(g_avg_wd)

reading existing grids
CPU times: user 1min 4s, sys: 13.3 s, total: 1min 17s
Wall time: 1min 17s


In [39]:
%%time
n_hotspot_cities, hotspot_stats_cities = hotspot_stats(g_avg_wd)

reading existing grids
CPU times: user 7.58 s, sys: 28 ms, total: 7.61 s
Wall time: 7.6 s


In [49]:
features = pd.concat([dv_cities, n_hotspot_cities, hotspot_stats_cities],axis=1)

In [52]:
crimes = pd.read_csv('data/qbased_crime_mexico.csv', index_col=0)

In [80]:
from scipy.stats import pearsonr, spearmanr, kendalltau

In [81]:
sig = {}
for c in features:
    pear_r, pear_p = pearsonr(features[c].values, crimes.total_crime.values)
    spea_r, spea_p = spearmanr(features[c].values, crimes.total_crime.values)
    kend_r, kend_p = kendalltau(features[c].values, crimes.total_crime.values)
    sig[c] = {'pear_r': pear_r, 'pear_p':pear_p, 'spea_r': spea_r, 'spea_p':spea_p,  'kend_r': kend_r, 'kend_p':kend_p}
sig = pd.DataFrame(sig).T

In [82]:
sig[sig.kend_p<0.1]

,kend_p,kend_r,pear_p,pear_r,spea_p,spea_r
nhot_20,0.078805,0.326363,0.079300,0.451325,0.089267,0.438558
nhot_21,0.094513,0.312261,0.145895,0.380569,0.123537,0.401182
n_med,0.086467,0.319339,0.226860,0.320056,0.116742,0.407953


In [54]:
import statsmodels.api as sm

In [57]:
features = features.loc[crimes.index]

In [112]:
Y = crimes.total_crime.values
X = n_hotspot_cities.loc[crimes.index].iloc[:,19:].values
# X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()

results.summary()

/home/Jiahui/miniconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     12.12
Date:                Thu, 10 Jan 2019   Prob (F-statistic):           0.000358
Time:                        00:00:07   Log-Likelihood:                 14.183
No. Observations:                  16   AIC:                            -18.37
Df Residuals:                      11   BIC:                            -14.50
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0068      0.013      0.542      0.599      -0.021       0.035
x2             0.0041      0.018      0.224      0.827      -0.036       0.044
x3            -0.0052      0.020     -0.264      0.797      -0.048       0.038
x4            -0.0180      0.029     -0.612      0.553      -0.083       0.047
x5             0.0191      0.020      0.971      0.352      -0.024       0.062
==============================================================================
Omnibus:                        0.051   Durbin-Watson:                   0.986
Prob(Omnibus):                  0.975   Jarque-Bera (JB):                0.196
Skew:                           0.109   Prob(JB):                        0.907
Kurtosis:                       2.504   Cond. No.                         208.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""